## Retrieve building coordinates

In [229]:
def get_address():
    
    import requests
    import geopandas
    from geopandas import GeoSeries
    from shapely.geometry import Polygon

    #Ask for user to input address
    #address = input("Enter Street:")
    
    #Example address
    #Eugeen de Bocklaan 14, 2900 Schoten
    street = 'Eugeen de bocklaan'
    nb = 14
    pc = 2900
    city = 'Schoten'
    
    global building_address
    building_address = street + str(nb) + city + str(pc)
    #Retrieve objectID for users address
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
    
    #if r.status_code is:
    #   continue
    #else: print("Please try again")
    
    objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]
    
    #Get building geometry
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()

    objectId = req["gebouw"]["objectId"]

    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()

    #Get building polygon coordinates
    global polygon
    polygon = [req["geometriePolygoon"]["polygon"]]
    
    #Convert polygon to more useful geopanda series
    t = []
    
    for i in polygon[0]['coordinates'][0]:
        t.append(tuple(i))
    
    global p
    p = Polygon(t)
    
    global g
    g = GeoSeries([p])
    
    global a
    #Area of the building
    a = g.area

    return polygon, p, g, a

get_address()

([{'coordinates': [[[156949.07425605506, 217250.54754476994],
     [156945.3703200519, 217245.2103287652],
     [156950.65044805408, 217241.68840876594],
     [156946.04539205134, 217235.10498476028],
     [156945.366096057, 217235.45480876043],
     [156943.5702560544, 217232.89314475656],
     [156947.74267205596, 217230.17224875465],
     [156947.96820805222, 217230.38652075827],
     [156951.59931205958, 217228.04092075303],
     [156953.36366406083, 217230.5961847566],
     [156952.82862406224, 217230.9778807573],
     [156953.97972805798, 217232.65199275687],
     [156956.4776480645, 217231.1953527555],
     [156958.00993606448, 217233.64322475716],
     [156959.8532640636, 217232.6118007563],
     [156964.66363206506, 217239.9182967618],
     [156959.2634400651, 217243.68482476473],
     [156958.82542406023, 217243.06799276546],
     [156955.34523206204, 217245.38888876513],
     [156955.66433606297, 217245.81064876541],
     [156949.07425605506, 217250.54754476994]]],
   'type'

## Get metadata of all raster files in Flanders

In [92]:
#Retrieve meta data for all raster files

def get_meta():
    
    #Get list of all geotiff files
    import os
    
    path = os.path.abspath('../')
    folder_path = path
    
    #Modify to select .tifs
    #Also modify to read DSM files together and DTM files together
    for path, dirs, files in os.walk(folder_path):
        for filename in files:
            f = open(os.path.join(path, filename))
            filelist = f.read()
    
    #For file in list read meta data
    import rasterio
    import pandas as pd
    
    #Make empty data frame
    
    #for file in filelist:
     #   with rasterio.open(file) as src:
      #      meta = src.meta()
       #     path = file
            
    #Append meta data to dataframe
    
    #Write df to a file
    
    return

get_meta()

SyntaxError: invalid syntax (<ipython-input-92-eb0eb38f927e>, line 3)

In [108]:
### Create list of all raster files
import os
    
path = os.path.abspath('/media/becode/GOPRO2/1DSM')
folder_path = path

filelist = []

for path, dirs, files in os.walk(folder_path):
        for filename in files:
            filelist.append(filename)
        print("There are ",len(filelist), "raster files")

There are  86 raster files


In [136]:
#Convert geotiff to cloud geotiff

from cogeotiff.cog import create_cog


src_path = "/media/becode/GOPRO2/1GEOTIFF/DHMVIIDTMRAS1m_k15.tif"
dest_path = "/media/becode/GOPRO2/1GEOTIFF/DHMVIIDTMRAS1m_k15.tif.cog"
create_cog(src_path, dest_path)

#the files are huge this is a bad idea

++++overview already exits, skip create overview++++

++++generate cloud optimized geotiff++++


In [217]:
### Create list of all raster files
import os
    
path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
folder_path = path

filelist = []

for path, dirs, files in os.walk(folder_path):
        for filename in files:
            filelist.append(filename)
        print("There are ",len(filelist), "raster files")
        
l = filelist

There are  15 raster files


In [179]:
filelist

['DHMVIIDSMRAS1m_k15.tif',
 'DHMVIIDTMRAS1m_k15.tif',
 'DHMVIIDSMRAS1m_k01.tif',
 'DHMVIIDSMRAS1m_k02.tif',
 'DHMVIIDSMRAS1m_k03.tif']

In [269]:
import rasterio
import pandas as pd

#Make an empty list

path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
boundslist = []
filenames = []
for f in filelist:
    filepath = os.path.join(path, f)
    filenames.append(f)
    with rasterio.open(filepath) as src:
        #metalist.append(src.meta())
        boundslist.append(src.bounds)
        
df = pd.DataFrame({'filelist': filenames,
     'bounds': boundslist})
            
    #Append meta data to dataframe
    
    #Write df to a file

In [270]:
df

,filelist,bounds
0,DHMVIIDSMRAS1m_k15.tif,"(130000.0, 198000.0, 162000.0, 218000.0)"
1,DHMVIIDTMRAS1m_k15.tif,"(130000.0, 198000.0, 162000.0, 218000.0)"
2,DHMVIIDSMRAS1m_k01.tif,"(145000.0, 238000.0, 162000.0, 247000.0)"
3,DHMVIIDSMRAS1m_k02.tif,"(162000.0, 238000.0, 194000.0, 250000.0)"
4,DHMVIIDSMRAS1m_k03.tif,"(194000.0, 238000.0, 206000.0, 248000.0)"
5,DHMVIIDSMRAS1m_k04.tif,"(53000.0, 218000.0, 66000.0, 225500.0)"
6,DHMVIIDSMRAS1m_k05.tif,"(66000.0, 218000.0, 98000.0, 232000.0)"
7,DHMVIIDSMRAS1m_k06.tif,"(98000.0, 218000.0, 130000.0, 227000.0)"
8,DHMVIIDSMRAS1m_k13.tif,"(66000.0, 198000.0, 98000.0, 218000.0)"
9,DHMVIIDSMRAS1m_k09.tif,"(194000.0, 218000.0, 226000.0, 238000.0)"


In [207]:
b = [box(l, b, r, t) for l, b, r, t in zip(df.left, df.bottom, df.right, df.top)]
boundslist[0]

BoundingBox(left=130000.0, bottom=198000.0, right=162000.0, top=218000.0)

## Search for raster file that contains the user address

In [205]:
#Search for correct geotiff file containing address

def address_raster():
    import pandas as pd
    df = pd.read_csv("address_raster")
    ##maybe use geopandas instead
    
    ##check which raster.bounds intersects with the address bounds
    
    ##return path to intersecting raster
    
    return path

## Calculate Digital Elevation Model for building

In [209]:
import rasterio
from rasterio.windows import Window
from rasterio.mask import mask
import numpy as np
import plotly.graph_objects as go

#File paths required
dsmfile = '/home/becode/dr-robin/theano/3D_houses_data/DHMVIIDSMRAS1m_k15.tif'
dtmfile = '/home/becode/dr-robin/theano/3D_houses_data/DHMVIIDTMRAS1m_k15.tif'

#Open DSM raster with mask of building shape
with rasterio.open(dsmfile) as src:
    mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
    #Read only pixels within the window/bounds of the building shape
    dsm = src.read(1, window = win)

#Open DTM raster with mask of building shape
with rasterio.open(dtmfile) as src:
    mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
    #Read only pixels within the window/bounds of the building shape
    dtm = src.read(1, window = win)

#Calculates raw digital elevation model (no resampling)
dem = dsm - dtm

#Calculates digital elevation model with resampling - for smoother building mesh

In [103]:
import rasterio
from rasterio.enums import Resampling

upscale_factor = 2

with rasterio.open(dsmfile) as src:

    mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
    #Read only pixels within the window/bounds of the building shape
    dsm = src.read(1, window = win).out_shape = (
        dsm.count,int(dsm.height * upscale_factor),
        int(dsm.width * upscale_factor),
        resampling = Resampling.bilinear)

    # scale image transform
    transform = dsm.transform * dsm.transform.scale(
        (dsm.width / data.shape[-1]),
        (dsm.height / data.shape[-2]))

SyntaxError: invalid syntax (<ipython-input-103-cefa5a71772d>, line 13)

In [210]:
#Show building height and floor area
height = round(dem.max(), 1)
print('The building height is:', height, 'meters')

area = round(int(a), 1)
print('The building floor area is:', area, 'sq meters')

The building height is: 8.4 meters
The building floor area is: 230 sq meters


In [230]:
import plotly.graph_objects as go

#Plot xyz of building
fig = go.Figure(data=[go.Surface(z=dem)])

#fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title=building_address, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 745-877-175
Debugger PIN: 745-877-175
Debugger PIN: 745-877-175
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
